In [19]:
import pandas as pd 
import plotly.express as px
import requests
# Set the default theme to plotly_dark
import plotly.io as pio
pio.templates.default = "plotly_dark"
import folium
from folium.plugins import MarkerCluster

import warnings
warnings.simplefilter('ignore')

# Map Locations to Map

In [21]:
df = pd.read_csv('data/raw_data/property_search.csv')
df

,name,beds,wc,garden,price,area_sft,location,leasehold,ctax_band,service_charge,agency,URL,reno_cost,yearly_rev
0,Purbrook Estate,3,1,0,475000,750,SE1 3DE,0,-,-,waterview,https://www.zoopla.co.uk/for-sale/details/6678...,25000,75700
1,Congreve Street,3,2,1,525000,897,SE17 1TG,85,c,-,foxtons,https://www.zoopla.co.uk/for-sale/details/6682...,20000,59600
2,Baltic Place,2,1,0,550000,833,N1 5AQ,112,d,3000,stirling ackroyd,https://www.rightmove.co.uk/properties/8644644...,5000,52800
3,Sutherland Avenue,2,1,1,550000,565,W9 2JD,0,-,-,dexters,https://www.rightmove.co.uk/properties/1459670...,10000,64400
4,Lancaster West,2,2,0,500000,668,W11 4AT,90,-,2200,winkworth,https://www.rightmove.co.uk/properties/1457131...,8000,69600
5,London Terrace Hoxton,2,1,0,480000,610,E2 7SQ,-,-,-,dexters,https://www.dexters.co.uk/property-for-sale/fl...,10000,54700
6,Royal Oak Court,2,1,1,550000,670,N1 6ES,84,b,1883,winkworth,https://www.winkworth.co.uk/properties/sales/r...,10000,67600
7,Maygood Street,2,1,0,450000,506,N1 0HB,959,-,-,winkworth,https://www.winkworth.co.uk/properties/sales/m...,25000,69900


In [22]:
base_url = 'https://api.postcodes.io/postcodes'
params = {
  "postcodes" : list(df.location.unique())
}

# Make a GET request to the API
response = requests.post(base_url, json=params).json()

lat_list = []
lon_list = []

for index, postcode in enumerate(df.location.unique()):
    lon_list.append(response['result'][index]['result']['longitude'])
    lat_list.append(response['result'][index]['result']['latitude'])

In [23]:
df['lat'] = lat_list
df['lon'] = lon_list
# df.to_csv('data/processed_data/processed_property_search.csv',index=False)
df.head()

,name,beds,wc,garden,price,area_sft,location,leasehold,ctax_band,service_charge,agency,URL,reno_cost,yearly_rev,lat,lon
0,Purbrook Estate,3,1,0,475000,750,SE1 3DE,0,-,-,waterview,https://www.zoopla.co.uk/for-sale/details/6678...,25000,75700,51.498920,-0.077847
1,Congreve Street,3,2,1,525000,897,SE17 1TG,85,c,-,foxtons,https://www.zoopla.co.uk/for-sale/details/6682...,20000,59600,51.491655,-0.085024
2,Baltic Place,2,1,0,550000,833,N1 5AQ,112,d,3000,stirling ackroyd,https://www.rightmove.co.uk/properties/8644644...,5000,52800,51.536710,-0.077481
3,Sutherland Avenue,2,1,1,550000,565,W9 2JD,0,-,-,dexters,https://www.rightmove.co.uk/properties/1459670...,10000,64400,51.524157,-0.188245
4,Lancaster West,2,2,0,500000,668,W11 4AT,90,-,2200,winkworth,https://www.rightmove.co.uk/properties/1457131...,8000,69600,51.512468,-0.215889


In [24]:
fig = px.scatter_mapbox(
    df,
    lat='lat',
    lon='lon',
    mapbox_style='carto-positron',
    zoom=11.5,
    hover_data={'name': True,
    'lat': False,
    'lon': False,
    'beds': True,
    'wc': True,
    'garden': True,
    'area_sft': True,
    'price': True,
    'agency': True,
    'reno_cost': True,
    'yearly_rev': True
    }  # Exclude lat and lon from hover
    )


fig.update_traces(marker=dict(size=12))

fig.update_layout(
    margin={"r":0,"t":40,"l":0,"b":0},
    title="Monk Property Tracker 🙉"
)
# fig.show(renderer="browser")
fig.show()

In [25]:
# Create a base map centered around a specific location
map_center = [df['lat'].mean(), df['lon'].mean()]  # Center the map based on the mean latitude and longitude
m = folium.Map(location=map_center, zoom_start=11.5)

# Create a MarkerCluster layer for better performance with large datasets
marker_cluster = MarkerCluster().add_to(m)

# Add markers for each property
for index, row in df.iterrows():
    # Construct the HTML content for the popup, including an image tag for each property
    popup_content = f"""
    <b>{row['name']}</b><br>
    Beds: {row['beds']}<br>
    WC: {row['wc']}<br>
    Garden: {row['garden']}<br>
    Area (sqft): {row['area_sft']}<br>
    Price: {row['price']}<br>
    Agency: {row['agency']}<br>
    Renovation Cost: {row['reno_cost']}<br>
    Yearly Revenue: {row['yearly_rev']}<br>
    <img src='https://picsum.photos/200/300', width="200"> 
    """

    # Create the marker with the customized popup content
    folium.Marker(
        location=[row['lat'], row['lon']],
        popup=popup_content,
    ).add_to(marker_cluster)

# Display the map
m.save('index.html')  # Save the map as an HTML file
m


### Append crime data

In [26]:
# combine list into tuples
combined_coords = list(zip(lat_list,lon_list))

In [27]:
crime_list = []
for location in combined_coords:
    lat = location[0]
    lon = location[1]
    url = f'https://data.police.uk/api/crimes-street/burglary?lat={lat}&lng={lon}'  # Corrected URL format
    responses = requests.get(url).json()
    crime_list.append(len(responses))  # Loop through unique names
    print('processed burglary count...')

df['burglary_count'] = crime_list

processed burglary count...
processed burglary count...
processed burglary count...
processed burglary count...
processed burglary count...
processed burglary count...
processed burglary count...
processed burglary count...


In [29]:
df['burglary_count']

0    10
1     0
2     0
3     0
4     0
5     0
6     5
7     0
Name: burglary_count, dtype: int64

In [30]:
def property_plot():    # Select numeric columns only
    numeric_df = df.select_dtypes(include=['float64', 'int64'])

    # Standardize the data
    df_standardized = (numeric_df - numeric_df.mean()) / numeric_df.std()
    df_standardized = df_standardized[['price','reno_cost','yearly_rev', 'burglary_count']]

    # Plot the standardized data
    fig = px.line(df_standardized, x=df.name, y=df_standardized.columns,
                labels={'value': 'Standardized Value', 'variable': 'Variable', 'index': 'Date'},
                title="Standardized Numeric Columns Over Time")

    return fig.show()

property_plot()